## ASSESSMENT 1
### To find
####  (3) Participant Daily level for Duration of MVPA (Moderate-Vigorous Physical Activity).
####  Total number of unique participants 604

In [1]:
import pandas as pd
import numpy as np
import re
from itertools import groupby
from operator import itemgetter
df=pd.read_csv('/Users/nsearch/Desktop/HPB_ASSESMENT/Assessment_Inputs/DS3_Heartrate_duration_intervals.csv')
df['ActivityDate'] = pd.to_datetime(df['ActivityDate'])
df.head()

,id,ActivityDate,MVPASession
0,ec296e97c0,2019-08-29,"[{""start"":451;""end"":459;""duration"":13};{""start..."
1,ec296e97c0,2019-08-27,"[{""start"":64;""end"":65;""duration"":6};{""start"":4..."
2,ec296e97c0,2019-08-26,"[{""start"":450;""end"":457;""duration"":12};{""start..."
3,ec296e97c0,2019-08-28,"[{""start"":447;""end"":456;""duration"":14};{""start..."
4,ec296e97c0,2019-01-09,"[{""start"":980;""end"":982;""duration"":7};{""start""..."


In [2]:

def getting_start_end_hour(input_text):
    
    each_session_obj=re.findall(r'\{(.*?)\}', input_text)
    
    cleaned_partcpnt_list=[]
    for each_sess in each_session_obj:
        
        dict_={}
        split_val=each_sess.split(';')
        #print(split_val)
        
        start=split_val[0].split(':')[1]
        end=split_val[1].split(':')[1]
        duration=split_val[2].split(':')[1]
        #print(start,end,duration)
        dict_['start']=int(start)
        dict_['end']=int(end)
        dict_['start_hour'] = ((int) (int(start)/(60)))%24 + 1
        dict_['end_hour']=((int) (int(end)/(60)))%24 + 1
        dict_['duration']=int(duration)
        cleaned_partcpnt_list.append(dict_)
        
    return cleaned_partcpnt_list

In [3]:
hour_end_time_in_min=np.arange(0, 1500, 60)

Partipnt_raw_text=list(df['MVPASession'])

def get_actual_mvpa_perhour(hour_end_time_in_min,Partipnt_raw_text):
    offset_dict=[]

    formatted_data=[]

    for each_partcpnt in Partipnt_raw_text:

        each_partcpnt=getting_start_end_hour(each_partcpnt)

        ##################

        addup_values=[]
        for start_time in each_partcpnt:

            addup_values.append(start_time['start_hour'])

        multiply_dict=dict(pd.Series(addup_values).value_counts())

        offset_dict.append(multiply_dict)

        ##################

        start_end_time=[]
        for each_session in each_partcpnt:


            if each_session['duration'] > 9:

                if each_session['start_hour']!=each_session['end_hour']:

                    for each_hour in hour_end_time_in_min:

                        if each_session['start'] <= each_hour <= each_session['end']:

                            first_hour_time=int(each_hour)-each_session['start']
                            second_hour_time=each_session['duration']-first_hour_time-5


                            tup1=(each_session['start_hour'],first_hour_time)
                            start_end_time.append(tup1)
                            tup2=(each_session['end_hour'],second_hour_time)
                            start_end_time.append(tup2)

                else:

                    matched_tup=(each_session['start_hour'],each_session['duration']-5)  
                    start_end_time.append(matched_tup)

        formatted_data.append(start_end_time)
    
    return offset_dict, formatted_data

        #print('Length of the formatted data',len(formatted_data))
        #print('Length of the offset dict',len(offset_dict))

offset_dict,formatted_data = get_actual_mvpa_perhour(hour_end_time_in_min,Partipnt_raw_text)


In [4]:
mapped_list=list(zip(formatted_data,offset_dict))
offset=5
def get_mvpa_per_hour(offset,mapped_list):
    hour_basis_mvpa=[]
    for each_set in mapped_list:
        result = dict([(k, sum(list(zip(*g))[1]) + each_set[1].get(k,0) * offset ) for k, g in groupby(each_set[0], itemgetter(0))])
        hour_basis_mvpa.append(result)
    return hour_basis_mvpa

hour_basis_mvpa=get_mvpa_per_hour(offset,mapped_list)
hour_basis_mvpa


[{8: 13,
  9: 11,
  10: 22,
  11: 33,
  12: 38,
  14: 23,
  15: 16,
  16: 17,
  17: 22,
  18: 13,
  19: 1,
  20: 29,
  21: 10},
 {8: 10, 13: 14, 16: 12, 17: 14, 18: 26, 19: 2, 21: 12},
 {8: 12, 11: 29, 12: 41, 13: 45, 14: 15, 15: 52, 16: 25, 18: 16, 19: 17},
 {8: 14,
  9: 16,
  10: 8,
  11: 11,
  12: 50,
  13: 19,
  14: 49,
  15: 36,
  16: 31,
  17: 43,
  18: 20,
  19: 9,
  20: 16,
  21: 17},
 {20: 24, 21: 14, 22: 13},
 {8: 16,
  10: 41,
  11: 20,
  12: 25,
  14: 31,
  15: 32,
  16: 18,
  17: 22,
  18: 29,
  19: 2,
  21: 10,
  23: 14},
 {},
 {20: 20, 21: 13},
 {21: 16, 22: 14},
 {18: 44, 19: 10, 20: 20},
 {},
 {22: 35, 23: 1},
 {15: 24, 24: 10},
 {19: 24, 20: 14, 23: 14},
 {15: 44, 16: 37, 17: 44, 18: 50, 19: 86, 21: 58, 22: 54, 23: 47},
 {13: 17,
  14: 1,
  15: 18,
  16: 11,
  17: 34,
  18: 29,
  19: 28,
  20: 12,
  21: 41,
  22: 39,
  23: 33},
 {13: 10},
 {19: 24, 20: 14, 23: 14},
 {},
 {},
 {7: 11, 18: 10, 19: 13, 20: 20},
 {8: 10},
 {10: 30, 11: 11, 15: 10},
 {10: 20, 11: 24, 15: 2

In [5]:
from collections import OrderedDict
keys=list(np.arange(1,25))
std_dict=dict((ele,0) for ele in keys)

Req_dict=[]
for i in hour_basis_mvpa:
    missing_keys = set(std_dict.keys()) - set(i.keys())
    for k in missing_keys:
        i[k] = std_dict[k]
    #s=OrderedDict(sorted(i.items(), key=lambda t: t[0]))
    Req_dict.append(i)
Req_dict

[{8: 13,
  9: 11,
  10: 22,
  11: 33,
  12: 38,
  14: 23,
  15: 16,
  16: 17,
  17: 22,
  18: 13,
  19: 1,
  20: 29,
  21: 10,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  13: 0,
  22: 0,
  23: 0,
  24: 0},
 {8: 10,
  13: 14,
  16: 12,
  17: 14,
  18: 26,
  19: 2,
  21: 12,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  14: 0,
  15: 0,
  20: 0,
  22: 0,
  23: 0,
  24: 0},
 {8: 12,
  11: 29,
  12: 41,
  13: 45,
  14: 15,
  15: 52,
  16: 25,
  18: 16,
  19: 17,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  9: 0,
  10: 0,
  17: 0,
  20: 0,
  21: 0,
  22: 0,
  23: 0,
  24: 0},
 {8: 14,
  9: 16,
  10: 8,
  11: 11,
  12: 50,
  13: 19,
  14: 49,
  15: 36,
  16: 31,
  17: 43,
  18: 20,
  19: 9,
  20: 16,
  21: 17,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  22: 0,
  23: 0,
  24: 0},
 {20: 24,
  21: 14,
  22: 13,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,


In [6]:
data=pd.DataFrame(Req_dict)
data=data.reindex(sorted(data.columns), axis=1)
new_columns=['Hr0'+str(i) if i <10 else 'Hr'+str(i) for i in np.arange(1,25)]
data=data.rename(columns=dict(zip(list(data.columns),new_columns)))
data['id']=df['id']
data=data.set_index('id')
data.insert(0,'ActivityDate',list(df['ActivityDate']))
data=data.replace(0,np.nan)
data

,ActivityDate,Hr01,Hr02,Hr03,Hr04,Hr05,Hr06,Hr07,Hr08,Hr09,...,Hr15,Hr16,Hr17,Hr18,Hr19,Hr20,Hr21,Hr22,Hr23,Hr24
id,,,,,,,,,,,,,,,,,,,,,
ec296e97c0,2019-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,...,16.0,17.0,22.0,13.0,1.0,29.0,10.0,NaN,NaN,NaN
ec296e97c0,2019-08-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,12.0,14.0,26.0,2.0,NaN,12.0,NaN,NaN,NaN
ec296e97c0,2019-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,...,52.0,25.0,NaN,16.0,17.0,NaN,NaN,NaN,NaN,NaN
ec296e97c0,2019-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,16.0,...,36.0,31.0,43.0,20.0,9.0,16.0,17.0,NaN,NaN,NaN
ec296e97c0,2019-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,24.0,14.0,13.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8180539542,2019-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,...,NaN,15.0,52.0,29.0,50.0,31.0,NaN,NaN,NaN,NaN
8180539542,2019-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,34.0,...,20.0,23.0,20.0,55.0,44.0,17.0,NaN,NaN,10.0,NaN
8180539542,2019-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,11.0,...,25.0,15.0,37.0,26.0,44.0,23.0,10.0,NaN,NaN,24.0


In [7]:
data.to_csv('/Users/nsearch/Desktop/HPB_ASSESMENT/Assessment_Inputs/DS3_Heartrate_duration_intervals_OUTPUT.csv')